In [2]:
import tweepy 
import pickle
import os
from geopy.geocoders import Nominatim
import countries
import datetime 
import time

In [3]:
TWEET_PATH = 'Tweets/'
ACCESS_TOKEN = '3007366663-QU3WM6hrAXEAfelPzdCpv713LOB8D7LgtsuvZWL'
ACCESS_SECRET = 'RzDTCkg3xoZfEmc3bGNXypponiq06ak9rZxiziXzx7nkO'
CONSUMER_KEY = 'nlbCSYMdqtyKpANbLQOl6ITKZ'
CONSUMER_SECRET = 'wY1CXDFLcN03H94BQo96KzKW47J2nfvoQBr4x5XL96POTV9Bbj'

auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_SECRET)

api = tweepy.API(auth)
api.wait_on_rate_limit = True
api.wait_on_rate_limit_notify = True

In [4]:
def get_all_tweets(user_id, api):


    alltweets = []
    #make initial request for most recent tweets (200 is the maximum allowed count)
    new_tweets = api.user_timeline(user_id = user_id,count=200)

    #save most recent tweets
    alltweets.extend(new_tweets)

    #save the id of the oldest tweet less one
    oldest = alltweets[-1].id - 1

    #keep grabbing tweets until there are no tweets left to grab
    while len(new_tweets) > 0:
        print "getting tweets before %s" % (oldest)

        #all subsiquent requests use the max_id param to prevent duplicates
        new_tweets = api.user_timeline(user_id = user_id, count=200, max_id=oldest)

        #save most recent tweets
        alltweets.extend(new_tweets)

        #update the id of the oldest tweet less one
        oldest = alltweets[-1].id - 1

        print "...%s tweets downloaded so far" % (len(alltweets))
    
    return alltweets


In [14]:
#all_tweets = get_all_tweets(40428817,api)

#for tweet in all_tweets:
    #print tweet.coordinates

In [15]:
#f = open('all_tweets.txt', 'w')
#pickle.dump(all_tweets, f)
#f.close()

#f = open('all_tweets.txt', 'r')
#all_tweets2 = pickle.load(f)
#f.close()


In [5]:
def get_followers(filename, screen_name, api):
    
    uscis_followers = []
    for page in tweepy.Cursor(api.followers_ids, screen_name = screen_name).pages():
        uscis_followers.extend(page)
        time.sleep(60)
    
    #write into a file
    f = open(filename , 'w')
    for id in uscis_followers:
        f.write(str(id) + '\n')
    f.close()
    
    return uscis_followers

In [17]:
#uscis_followers = get_followers('uscis_followers','USCIS' ,api)

In [7]:
f = open('uscis_followers', 'r')
uscis_followers = []
for line in f:
    uscis_followers.append(f.read)
f.close()
print len(uscis_followers)

65996


In [8]:
def prune_followers(uscis_followers, min_status_count, api):
    
    pruned = []
    
    for id in uscis_followers:
        try:
            user = api.get_user(id)
        
            if not user.protected:
                if user.geo_enabled:
                    if user.statuses_count > min_status_count:
                        pruned.append(id)
                        print id
        except:
            print 'error caught!'
            pass
    
    return pruned

In [ ]:
pruned_followers1 = prune_followers(uscis_followers[0:6000], 100, api)

In [ ]:
#pruned_followers2 = prune_followers(uscis_followers[6000:12000], 100, api)

In [ ]:
#pruned_followers3 = prune_followers(uscis_followers[12000:18000], 100, api)

In [ ]:
#pruned_followers4 = prune_followers(uscis_followers[18000:24000], 100, api)

In [ ]:
#pruned_followers5 = prune_followers(uscis_followers[24000:30000], 100, api)

In [ ]:
#pruned_followers6 = prune_followers(uscis_followers[30000:36000], 100, api)

In [ ]:
#pruned_followers7 = prune_followers(uscis_followers[36000:42000], 100, api)

In [ ]:
#pruned_followers8 = prune_followers(uscis_followers[42000:48000], 100, api)

In [ ]:
#pruned_followers9 = prune_followers(uscis_followers[48000:54000], 100, api)

In [ ]:
#pruned_followers10 = prune_followers(uscis_followers[54000:60000], 100, api)

In [ ]:
#pruned_followers11 = prune_followers(uscis_followers[60000:], 100, api)

In [31]:
def get_and_save_tweets_one_user(user_id, api):
    all_tweets = get_all_tweets(user_id, api)
    f = open (TWEET_PATH + str(user_id) , 'w')
    pickle.dump(all_tweets, f)
    f.close

In [32]:
def get_and_save_tweets_all_users(followers_pruned_id, api):
    for id in followers_pruned_id:
        get_and_save_tweets_one_user(id, api)
        print 'tweets of ' , id , 'were saved!' 

In [33]:
#get_and_save_tweets_all_users(pruned_followers, api)

In [9]:
def get_user_ids_Twetter_folder():
    all_followers = []
    for dirs, root, files in os.walk(TWEET_PATH):
        for file in files:
            if not file.startswith('.'):
                all_followers.append(file) 
    return all_followers

In [34]:
#all_followers = get_user_ids_Twetter_folder()

In [77]:
class User_Tweet(object):

    def __init__(self,date,text,country):
        self.date = date
        self.text = text
        self.country = country
    
    def __cmp__(self,other):
        if self.date < other.date:
            return -1
        elif self.date == other.date:
            return 0
        else:
            return 1

In [87]:
def create_user_timelines_dic(followers_ids):
    
    users_timelines = {}
    
    for user_id in followers_ids:
        
        timeline = []
        
        file_name = TWEET_PATH + str(user_id)
        f = open(file_name,'r')
        tweets = pickle.load(f)
        f.close()
    
        for tweet in tweets:
            if tweet.place is not None:
                user_tweet = User_Tweet(tweet.created_at,tweet.text,tweet.place.country)
                timeline.append(user_tweet)
                
        users_timelines[user_id] = timeline
        
    return users_timelines

In [88]:
users_timelines = create_user_timelines_dic(all_followers)

In [35]:
def find_immigrants(users_timelines):
    
    immigrants = {}
    
    for user_id in users_timelines:
        timeline = users_timelines[user_id]
        sorted_timeline = sorted(timeline)
    
        countries = []
        for user_tweet in sorted_timeline:
            if user_tweet.country not in countries:
                countries.append(user_tweet.country)
            
        if (len(countries) > 1):
            if sorted_timeline[-1].country == countries[-1]:

                #find the time of last tweet in the previous country
                i = len(sorted_timeline) - 1
                last_tweet_previous_country = sorted_timeline[i]
                while(last_tweet_previous_country.country != countries[-2]):
                    i = i - 1
                    last_tweet_previous_country = sorted_timeline[i]
                
                duration = sorted_timeline[-1].date - last_tweet_previous_country.date
            
                if duration.days > 90:
                    immigrants[user_id] = duration.days
    return immigrants             

In [ ]:
immigrants = find_immigrants(users_timelines)